In [1]:
import os

In [2]:
os.getcwd()

'/home/bhvignesh/trading_system/notebooks'

In [1]:
# trading_system/main.py

import src.collectors.data_collector as data_collector
import pandas as pd

tickers = pd.read_excel("data/ticker.xlsx")
tickers = tickers[~tickers["Security Name"].duplicated()]
tickers.reset_index(drop=True, inplace=True)

def add_ticker_suffix(x):
    if x["Exchange"]=="BSE":
        return x["Security Name"] + ".BO"
    else:
        return x["Security Name"] + ".NS"

tickers["Ticker"] = tickers.apply(add_ticker_suffix, axis = 1)
all_tickers = tickers["Ticker"].tolist()

# all_tickers = all_tickers[:25]

print(all_tickers)

data_collector.main(all_tickers)





2025-03-02 05:24:03,832 - src.database.engine - INFO - Database connection established successfully
2025-03-02 05:24:03,833 - src.collectors.data_collector - INFO - Refreshing data for tickers: ['LODHA.NS', 'ATUL.BO', 'ICICIPRULI.NS', 'FINEORG.BO', 'KOTAKBANK.BO', 'CANBK.BO', 'WIPRO.BO', 'JSWINFRA.BO', 'MEDPLUS.BO', 'TATACONSUM.BO', 'INDIACEM.BO', 'RRKABEL.BO', 'POLYMED.BO', 'CANFINHOME.BO', 'LINDEINDIA.BO', 'GLAND.BO']


['HDFCBANK.BO', 'TCS.BO', 'LT.BO', 'INFY.BO', 'RELIANCE.BO', 'ZOMATO.BO', 'TATAMOTORS.BO', 'ICICIBANK.BO', 'SBIN.BO', 'HCLTECH.BO', 'ITC.BO', 'M&M.BO', 'BAJFINANCE.BO', 'INDUSINDBK.BO', 'BHARTIARTL.BO', 'HINDUNILVR.BO', 'POWERGRID.BO', 'ASIANPAINT.BO', 'TECHM.BO', 'TITAN.BO', 'MARUTI.BO', 'BAJAJFINSV.BO', 'TATASTEEL.BO', 'NTPC.BO', 'NESTLEIND.BO', 'ADANIPORTS.BO', 'ULTRACEMCO.BO', 'AXISBANK.BO', 'KOTAKBANK.BO', 'SUNPHARMA.BO', 'CIPLA.BO', 'SUZLON.BO', 'JIOFIN.BO', 'ADANIENT.BO', 'ADANIGREEN.BO', 'RECLTD.BO', 'PFC.BO', 'BAJAJ-AUTO.BO', 'YESBANK.BO', 'DMART.BO', 'BEL.BO', 'PNB.BO', 'BANKBARODA.BO', 'WIPRO.BO', 'HINDALCO.BO', 'COALINDIA.BO', 'CANBK.BO', 'TATAPOWER.BO', 'SRF.BO', 'PERSISTENT.BO', 'ADANIPOWER.BO', 'HAL.BO', 'LTIM.BO', 'IOC.BO', 'VEDL.BO', 'ONGC.BO', 'SHRIRAMFIN.BO', 'BPCL.BO', 'DABUR.BO', 'GAIL.BO', 'DRREDDY.BO', 'SBILIFE.BO', 'CUMMINSIND.BO', 'HEROMOTOCO.BO', 'AMBUJACEM.BO', 'TRENT.BO', 'AUBANK.BO', 'INDIGO.BO', 'NAUKRI.BO', 'MARICO.BO', 'TVSMOTOR.BO', 'MAXHEALTH.BO', 'DLF

2025-03-02 05:24:05,054 - src.collectors.base_collector - INFO - Created table daily_prices with IF NOT EXISTS
2025-03-02 05:24:05,538 - src.collectors.base_collector - INFO - Successfully saved 958 rows to daily_prices
2025-03-02 05:24:05,577 - src.collectors.price_collector - INFO - Successfully refreshed data for LODHA.NS in daily_prices.
2025-03-02 05:24:05,655 - src.collectors.base_collector - INFO - Deleted existing data for KOTAKBANK.BO in daily_prices
2025-03-02 05:24:05,679 - src.collectors.base_collector - INFO - Deleted existing data for ATUL.BO in daily_prices
2025-03-02 05:24:06,960 - src.collectors.base_collector - INFO - Successfully saved 2080 rows to daily_prices
2025-03-02 05:24:06,962 - src.collectors.price_collector - INFO - Successfully refreshed data for ICICIPRULI.NS in daily_prices.
2025-03-02 05:24:08,268 - src.collectors.base_collector - INFO - Successfully saved 6198 rows to daily_prices
2025-03-02 05:24:08,269 - src.collectors.price_collector - INFO - Succes

In [2]:
# test_williams_r.py

import logging
from src.database.config import DatabaseConfig
from src.strategies.mean_reversion.williams_percent_r_start import WilliamsRStrategy

# Configure logging to see debug/info output
logging.basicConfig(level=logging.DEBUG, format="%(asctime)s - %(levelname)s - %(message)s")

def main():
    # Create the default database configuration.
    db_config = DatabaseConfig.default()

    # Define strategy-specific parameters.
    params = {
        "wr_period": 14,                 # Lookback period for WR calculation.
        "oversold_threshold": -80,       # Oversold threshold for buy signal.
        "overbought_threshold": -20,     # Overbought threshold for sell signal.
        "data_lookback": 252,            # Fallback number of records if no date range is provided.
        "long_only": True,               # Long-only strategy.
        "stop_loss_pct": 0.05,           # Stop loss percentage.
        "take_profit_pct": 0.10,         # Take profit percentage.
        "slippage_pct": 0.001,           # Slippage percentage.
        "transaction_cost_pct": 0.001    # Transaction cost percentage.
    }

    # Instantiate the Williams %R strategy.
    strategy = WilliamsRStrategy(db_config, params)

    try:
        # Test 1: End-of-Day (EOD) forecast: only the latest signal.
        print("=== EOD Forecasting Signal for RELIANCE.BO ===")
        latest_signal_df = strategy.generate_signals("RELIANCE.BO", latest_only=True)
        print(latest_signal_df)

        # Test 2: Backtest over a specific date range.
        print("=== Backtesting (2023) for RELIANCE.BO ===")
        backtest_df = strategy.generate_signals(
            "RELIANCE.BO",
            start_date="2023-01-01",
            end_date="2023-12-31",
            latest_only=False
        )
        print(backtest_df.head())

    except Exception as e:
        logging.error("Error running WilliamsRStrategy: %s", e)

if __name__ == "__main__":
    main()

2025-03-05 17:24:22,159 - INFO - Database connection established successfully
2025-03-05 17:24:22,189 - INFO - WilliamsRStrategy initialized with parameters: {'wr_period': 14, 'oversold_threshold': -80, 'overbought_threshold': -20, 'data_lookback': 252, 'long_only': True, 'stop_loss_pct': 0.05, 'take_profit_pct': 0.1, 'slippage_pct': 0.001, 'transaction_cost_pct': 0.001}


=== EOD Forecasting Signal for RELIANCE.BO ===


2025-03-05 17:24:33,506 - DEBUG - Executed query in 11.2729 seconds
2025-03-05 17:24:33,536 - DEBUG - get_historical_prices executed in 11.3360 seconds


              open         high          low        close  volume         wr  \
date                                                                           
2025-02-28  1201.0  1217.400024  1193.300049  1199.599976  437656 -91.334345   

            signal  signal_strength exit_type  return  position  \
date                                                              
2025-02-28       0        71.334345      none     0.0       0.0   

            cumulative_return  
date                           
2025-02-28                0.0  
=== Backtesting (2023) for RELIANCE.BO ===


2025-03-05 17:24:34,002 - DEBUG - Executed query in 0.3054 seconds
2025-03-05 17:24:34,007 - DEBUG - get_historical_prices executed in 0.3113 seconds
2025-03-05 17:24:34,049 - DEBUG - Database engine disposed successfully


                   open         high          low        close  volume  \
date                                                                     
2023-01-19  1220.079750  1223.408316  1211.844655  1218.551147  403116   
2023-01-20  1223.778102  1223.778102  1202.278103  1204.546387  348024   
2023-01-23  1208.146166  1216.036100  1196.311265  1198.086548  427720   
2023-01-24  1199.516511  1205.138040  1177.893112  1190.960815  698698   
2023-01-25  1188.421218  1190.788222  1174.120714  1175.082275  207828   

                   wr  signal  signal_strength    exit_type    return  \
date                                                                    
2023-01-19 -78.672056       0        58.672056         none  0.000000   
2023-01-20 -95.358473       1        15.358473  signal_exit -0.003992   
2023-01-23 -97.988772       0        77.988772  signal_exit -0.009333   
2023-01-24 -87.751339       0        67.751339         none  0.000000   
2023-01-25 -99.129487       0        79.129

In [3]:
import os

def get_all_files(directory):
    all_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            all_files.append(os.path.join(root, file))
    return all_files


In [4]:
os.getcwd()

'/home/bhvignesh/trading_system'

In [5]:
get_all_files('/home/bhvignesh/trading_system')

['/home/bhvignesh/trading_system/poetry.lock',
 '/home/bhvignesh/trading_system/__init__.py',
 '/home/bhvignesh/trading_system/.env',
 '/home/bhvignesh/trading_system/requirements.txt',
 '/home/bhvignesh/trading_system/.gitignore',
 '/home/bhvignesh/trading_system/README.md',
 '/home/bhvignesh/trading_system/main.py',
 '/home/bhvignesh/trading_system/pyproject.toml',
 '/home/bhvignesh/trading_system/tests/Claude_testCase.py',
 '/home/bhvignesh/trading_system/tests/__init__.py',
 '/home/bhvignesh/trading_system/tests/database/test_config.py',
 '/home/bhvignesh/trading_system/tests/database/__init__.py',
 '/home/bhvignesh/trading_system/tests/database/test_engine.py',
 '/home/bhvignesh/trading_system/tests/collectors/test_base_collector.py',
 '/home/bhvignesh/trading_system/tests/collectors/test_statements_collector.py',
 '/home/bhvignesh/trading_system/tests/collectors/test_info_collector.py',
 '/home/bhvignesh/trading_system/tests/collectors/__init__.py',
 '/home/bhvignesh/trading_syst

In [2]:
db_config = DatabaseConfig.default()

# Define strategy-specific parameters.
params = {
    "wr_period": 14,                 # Lookback period for WR calculation.
    "oversold_threshold": -80,       # Oversold threshold for buy signal.
    "overbought_threshold": -20,     # Overbought threshold for sell signal.
    "data_lookback": 252,            # Fallback number of records if no date range is provided.
    "long_only": True,               # Long-only strategy.
    "stop_loss_pct": 0.05,           # Stop loss percentage.
    "take_profit_pct": 0.10,         # Take profit percentage.
    "slippage_pct": 0.001,           # Slippage percentage.
    "transaction_cost_pct": 0.001    # Transaction cost percentage.
}

# Instantiate the Williams %R strategy.
strategy = WilliamsRStrategy(db_config, params)

2025-03-05 16:16:31,221 - INFO - Database connection established successfully
2025-03-05 16:16:31,223 - INFO - WilliamsRStrategy initialized with parameters: {'wr_period': 14, 'oversold_threshold': -80, 'overbought_threshold': -20, 'data_lookback': 252, 'long_only': True, 'stop_loss_pct': 0.05, 'take_profit_pct': 0.1, 'slippage_pct': 0.001, 'transaction_cost_pct': 0.001}


In [3]:
latest_signal_df = strategy.generate_signals("RELIANCE.BO", latest_only=True)

ValueError: Wrong number of arguments for _execute_query(self, query: Any, params: dict, index_col: Optional[str] = None) -> pandas.core.frame.DataFrame:
     _execute_query(<sqlalchemy.sql.elements.TextClause object at 0x7f6122b87890>, {'ticker': 'RELIANCE.BO', 'data_source': 'yfinance', 'lookback': 15}, index_col=date) was called.